In [ ]:
import numpy as np
import optuna
from hftbacktest import BacktestAsset, HashMapMarketDepthBacktest
from hftbacktest import Recorder
from hftbacktest.stats import LinearAssetRecord
from strategies.glft import gridtrading_glft_mm

# Data Input paths
preprocessed_data_path = "../data/daily_processed"
daily_eod_snapshots = "../data/snapshots"

In [ ]:
# Taken from https://support.deribit.com/hc/en-us/articles/25944746248989-Fees
MAKER_FEE = -0.0001   
TAKER_FEE =  0.0005    

max_position=50

In [ ]:
day_start = 2
day_end = 22
n_trading_days = day_end - day_start

def objective(trial):
    data_paths = []
    for i in range(day_start, day_end):
        day_file = f"../data/daily_processed/deribit_eth_perp_2025-01-{i:02d}.npz"
        data_paths.append(day_file)
        
    eod = np.load(f"../data/snapshots/deribit_eth_perp_2025-01-{day_start:02d}_eod.npz")['data']

    # Hyperparameters under optimization
    gamma = trial.suggest_float("gamma", 0.01, 0.1)
    delta = trial.suggest_int("delta", 1, 10)
    adj1 = trial.suggest_float("adj1", 0.01, 2)
    adj2 = trial.suggest_float("adj2", 0.01, 1)

    asset = (
        BacktestAsset()
            .data(data_paths)
            .initial_snapshot(eod)
            .linear_asset(1.0)
            .constant_latency(10000, 10000) 
            .risk_adverse_queue_model()
            .no_partial_fill_exchange()
            .trading_value_fee_model(MAKER_FEE, TAKER_FEE)
            .tick_size(0.01)
            .lot_size(1)
            .last_trades_capacity(10000))
    
    hbt = HashMapMarketDepthBacktest([asset])
    recorder = Recorder(1, n_trading_days* 1_000_000)
    
    gridtrading_glft_mm(hbt, recorder.recorder, n_trading_days, gamma, delta, adj1, adj2, max_position)
    hbt.close()

    stats = LinearAssetRecord(recorder.get(0)).stats()
    return stats.summary(pretty=True)['Return'][0]

In [ ]:
from optuna.samplers import TPESampler

sampler = TPESampler(seed=42)
study = optuna.create_study(direction='maximize', sampler=sampler)
study.optimize(objective, n_trials=100)

In [ ]:
print("BEST Parameter Combinations", study.best_params)
print("BEST Return Achieved", study.best_value)